In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as fn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import shuffle
from sklearn import preprocessing 
%matplotlib inline
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

Device is cuda!


In [35]:
import pandas as pd

df = pd.read_csv('./Train_Test_Network.csv')
df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [36]:
df = df[['conn_state', 'proto', 'dns_rejected', 'service', 'dns_AA', 'dns_qtype', 'dns_rcode', 'dns_query', 'dns_RA', 'dns_RD', 'dst_ip_bytes', 'src_pkts', 'label', 'type']]
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)
df.isna().sum()
print(df['type'].value_counts())
df.head()

type
normal        40415
xss            3169
ddos            734
injection       669
dos             584
mitm            571
password        276
scanning         75
backdoor         47
ransomware       24
Name: count, dtype: int64


,conn_state,proto,dns_rejected,service,dns_AA,dns_qtype,dns_rcode,dns_query,dns_RA,dns_RD,dst_ip_bytes,src_pkts,label,type
405545,S1,tcp,-,http,-,0,0,-,-,-,3786,6,0,normal
287784,SF,udp,F,dns,T,12,0,220.5.70.168.in-addr.arpa,T,F,172,1,0,normal
387208,SF,udp,F,dns,F,12,3,3.3.e.c.4.8.f.f.1.0.0.0.0.0.0.0.0.0.0.0.0.0.0....,F,F,800,1,0,normal
270500,SF,udp,F,dns,F,43,0,235.135.in-addr.arpa,F,F,525,1,0,normal
254871,SF,udp,F,dns,F,43,0,170.166.in-addr.arpa,F,F,408,1,0,normal


In [37]:
label_encoder = preprocessing.LabelEncoder() 
df['dns_query']= label_encoder.fit_transform(df['dns_query']) 

In [38]:
df.replace('-', -1, inplace=True)
df = df.dropna(axis=1, how='all')
df.head()

,conn_state,proto,dns_rejected,service,dns_AA,dns_qtype,dns_rcode,dns_query,dns_RA,dns_RD,dst_ip_bytes,src_pkts,label,type
405545,S1,tcp,-1,http,-1,0,0,2,-1,-1,3786,6,0,normal
287784,SF,udp,F,dns,T,12,0,4765,T,F,172,1,0,normal
387208,SF,udp,F,dns,F,12,3,6112,F,F,800,1,0,normal
270500,SF,udp,F,dns,F,43,0,5259,F,F,525,1,0,normal
254871,SF,udp,F,dns,F,43,0,2790,F,F,408,1,0,normal


In [39]:
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x].astype(int)
    df.drop(name, axis=1, inplace=True)
    
encode_text_dummy(df,'conn_state')
encode_text_dummy(df,'proto')
encode_text_dummy(df,"dns_rejected")
encode_text_dummy(df,"service")
encode_text_dummy(df,"dns_AA")
encode_text_dummy(df,"dns_RA")
encode_text_dummy(df,"dns_RD")

In [40]:
df

,dns_qtype,dns_rcode,dns_query,dst_ip_bytes,src_pkts,label,type,conn_state-OTH,conn_state-REJ,conn_state-RSTO,...,service-ssl,dns_AA--1,dns_AA-F,dns_AA-T,dns_RA--1,dns_RA-F,dns_RA-T,dns_RD--1,dns_RD-F,dns_RD-T
405545,0,0,2,3786,6,0,normal,0,0,0,...,0,1,0,0,1,0,0,1,0,0
287784,12,0,4765,172,1,0,normal,0,0,0,...,0,0,0,1,0,0,1,0,1,0
387208,12,3,6112,800,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
270500,43,0,5259,525,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
254871,43,0,2790,408,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254576,1,0,12284,158,1,0,normal,0,0,0,...,0,0,0,1,0,1,0,0,1,0
229743,0,0,2,8462,10,1,ddos,0,0,0,...,1,1,0,0,1,0,0,1,0,0
247161,12,0,3631,375,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
293839,28,0,9338,712,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [41]:
df.to_csv('preprocessed.csv', encoding='utf-8', index=False)